#### Introduction

This is a mini project for SC1003 Introduction to Computation Thinking and Programming in NTU. We are from FCSD-Group 4 consisting of Yan Jie, Samuel, Timothy, Keith and Run Ze.

#### Algorithmic Thinking
The program is a software that aims to handle large csv files containing student details of different tutorial groups and sort them accordingly into even and distributed teams based on their, in order of significance:
1. Average CGPA
2. Faculty
3. Gender

#### Flowchart

#### Libraries and Dependencies

In [ ]:
# Import visualization libraries (Graphing and Analysis only)
import plotly.graph_objects as go

# Random library to generate random integers (Mainly used for sorting)
import random

# Additional Requirements widgets library
import ipywidgets as widgets
from IPython.display import display

#### Basic File Management and Sorting Functions

In [ ]:
# We have decided to integrate a very basic Object Oriented Design consideration into our sorting algorithm to make it more intuitive 
class Student:
    # These are all the init properties that align with the csv file headers
    def __init__(self, tgrp, id, faculty, name, gender, cgpa):
        self.tutorialGrp = int(tgrp)
        self.studID = id
        self.faculty = faculty
        self.name = name
        self.gender = gender
        self.cgpa = float(cgpa)
    
    # A print output method for debugging or general use purposes
    def printStudent(self):
        print(f'{self.tutorialGrp}, {self.studID}, {self.faculty}, {self.name}, {self.gender}, {self.cgpa}, {self.team}')

In [ ]:
# Using IPywidgets to introduce interactivity for File Management
'''
import codecs as codecs

fileInput = widgets.FileUpload(
    accept='.csv,.txt',
    multiple=False,
    error='Wrong file type! Please ensure it is a csv or txt file'
)

# Display the file upload button
display(fileInput)

# Retrieves the file's contents
uploadedFile = fileInput.value[0]['content']
convertedFile = codecs.decode(uploadedFile, encoding='utf-8')
'''

In [ ]:
# Initialize data with 2 global scope lists
studentList = []
tgrpsTeamsList = {}

# File management code to open, read and save the records locally
with open('records.csv') as records:
    records.__next__() # Skip the header row
    lines = records.read().strip().split('\n') # We skip the header row using .read()
    
    # Loop through each row in the csv file
    for line in lines:
        line = line.split(',') # Seperate each column item in a row by using split
        line[0] = line[0][2:] # Here we do an additional change to the tutorial group by removing the "G-" segment of the string to allow for a more intuitive sort within the code
        
        student = Student(line[0], line[1], line[2], line[3], line[4], line[5]) # Then we create each individual student object
        studentList.append(student) # And lastly save each student object to the global scope studentList

In [ ]:
# We use lambda to denote the key i.e., the columns of the csv / object property that we want to sort our rows of students by
# The anonymous function allows us to efficiently "retrieve" the intended sorting conditions for sorting the student list

# The reason why we stripped the tutorial groups from "G-1" to "1" is so that we can utilize the sorted function to efficiently create a pre-sorted
# data set which reduces the overall runtime / time complexity of the later sorting algorithms.

# Sort dataList function by tutorial group
def sortData(data):
    sortedData = sorted(data, key=lambda Student: Student.tutorialGrp)
    return sortedData

# Sort dataList by tutorial group and CGPA
def sortCGPA(data):
    # The sort is not in reverse order, therefore it will show the cgpa from lowest to highest
    sortedData = sorted(data, key=lambda Student: (Student.tutorialGrp, Student.cgpa))
    return sortedData

# A function that counts and returns the total number of unique tutorial groups
def getNumOfTgrps(studentList):
    tgrpList = []
    
    # Loop through the student list and check if the tutorial group exists in the tgrp list, if not then append the tutorial group in e.g., "1" / "2"
    for student in studentList:
        if student.tutorialGrp not in tgrpList:
            tgrpList.append(student.tutorialGrp)
    
    return len(tgrpList)

# A function solely used for debugging
def printTeam(team):
    for student in team:
        print(f'{student.name} and {student.cgpa}\n')
        
# A function for outputting all the teams from each tutorial group in a beautified formatted print statement
def printGrpsAndTeams():
    for tgrps in tgrpsTeamsList:
        print(f'\n-------------- Tutorial Group {tgrps} --------------\n')
        for teams in tgrpsTeamsList[tgrps]:
            print(f'--------- Team {tgrpsTeamsList[tgrps].index(teams) + 1} ---------')
            totalCGPA = 0.00
            num, male, female = 0, 0, 0
            faculties = {}
            
            for student in teams:
                totalCGPA += student.cgpa
                num += 1
            
                if student.gender == "Male":
                    male += 1
                else:
                    female += 1

                faculties[student.faculty] = 0
                faculties[student.faculty] += 1

                print(f'{student.name} and {student.gender[:1]} and {student.faculty}')

            print(f'\n{totalCGPA / num:.2f}')
            print(f'Male:{male} Female: {female}')
            print(f'Unique Number of Faculties: {len(faculties)}\n')

#### Data Analysis with Graphs from Plotly

In [ ]:
# Graph Visualization functions using Plotly
def showCGPAPerGroup():
    cgpa = []
    tgrp = []
    for student in sortCGPA(studentList):
        cgpa.append(student.cgpa)
        tgrp.append(student.tutorialGrp)

    # Make it into a readable box plot using the python arrays above
    boxTrace = go.Box(
        x = tgrp,
        y = cgpa,
        name = 'CGPA / Tutorial Group'
    )

    # Initialize the graph object with go.Figure()
    fig = go.Figure(boxTrace)
    fig.update_layout(
        title="CGPA / Tutorial Group",
        xaxis_title="Tutorial Group",
        yaxis_title="CGPA") # Name the title of the graph
    fig.show() # Show the graph
    
# showCGPAPerGroup()

In [ ]:
def showStudentsPerGroup():
    # Graph on No. of Students Per Tutorial Group
    numPerGrp = {}
    studList = sortData(studentList)

    for student in studList:
        if student.tutorialGrp not in numPerGrp.keys():
            numPerGrp[student.tutorialGrp] = 0

    for updateStudent in studList:
        numPerGrp[updateStudent.tutorialGrp] += 1

    barTrace = go.Bar(
        x = list(numPerGrp.keys()),
        y = list(numPerGrp.values()),
        name = 'Number of Students / Tutorial Group'
    )

    # Initialize the graph object with go.Figure()
    fig = go.Figure(barTrace)
    fig.update_layout(
        title="No. of Students / Tutorial Group",
        xaxis_title="Tutorial Group",
        yaxis_title="No. Of Students") # Name the title of the graph
    fig.show() # Show the graph

# showStudentsPerGroup()

In [ ]:
def showFacultyPerGroup(tutorialGrpNum):
    # Graph on Faculty Distribution per Tutorial Group
    facPerGrp = {}
    numPerGrp = {}
    studList = sortData(studentList)

    # Get the number of students per tutorial group
    for student in studList:
        if student.tutorialGrp not in numPerGrp.keys():
            numPerGrp[student.tutorialGrp] = 0
                
        if student.faculty not in facPerGrp.keys():
            facPerGrp[student.faculty] = 0

    for updateStudent in studList:
        numPerGrp[updateStudent.tutorialGrp] += 1

    # Input the tutorial group number to show the distribution for that group
    tgrpToShow = tutorialGrpNum

    # Get the number of students per faculty in a tutorial group
    for i in range(numPerGrp[tgrpToShow] * tgrpToShow - 50,
                numPerGrp[tgrpToShow] * tgrpToShow):
        facPerGrp[studList[i].faculty] += 1 # incremement the facPerGrp dict by referencing the indexed student and their faculty

    barTrace1 = go.Bar(
            x = list(facPerGrp.keys()),
            y = list(facPerGrp.values()),
            name = 'No. of Students / Faculty'
        )

    # Initialize the graph object with go.Figure()
    fig = go.Figure(barTrace1)
    fig.update_layout(
        title="No. of Students / Faculty",
        xaxis_title="Faculty",
        yaxis_title="No. Of Students") # Name the title of the graph
    fig.show() # Show the graph
    
# showFacultyPerGroup(120)

In [ ]:
def showGenderPerGroup():
    # Graph on Gender Count per Tutorial Group 
    tgrp = []
    malePerGrp = {}
    femalePerGrp = {}
    studList = sortData(studentList)

    for student in studList:
        if student.tutorialGrp not in tgrp:
            tgrp.append(student.tutorialGrp)
            malePerGrp[student.tutorialGrp] = 0
            femalePerGrp[student.tutorialGrp] = 0
        
    for updateNum in studList:
        if updateNum.gender == 'Male':
            malePerGrp[updateNum.tutorialGrp] += 1
        if updateNum.gender == 'Female':
            femalePerGrp[updateNum.tutorialGrp] += 1

    maleTrace = go.Bar(
        x = tgrp,
        y = list(malePerGrp.values()),
        hovertext='(Tutorial Group, No. Of Males)',
        name = 'Male')

    femaleTrace = go.Bar(
        x = tgrp,
        y = list(femalePerGrp.values()),
        hovertext = '(Tutorial Group, No. Of Females)',
        name = 'Female')

    # Initialize the graph object with go.Figure()
    fig = go.Figure()
    fig.add_trace(maleTrace)
    fig.add_trace(femaleTrace)
    fig.update_layout(
        title = "Gender / Tutorial Group", # Name the title of the graph
        xaxis_title = "Tutorial Group",
        yaxis_title = "No. Of Students")
    fig.show() # Show the graph

# showGenderPerGroup()

#### First Algorithm

In [ ]:
# A function to retrieve and return the sorted specified Tutorial Group by gpa
def retrieveTutorialGroup(studentList, groupNum):
    studList = sortCGPA(studentList)
    tgrpList = []
    for student in studList:
        if student.tutorialGrp == groupNum:
            tgrpList.append(student)
    return tgrpList

# A function to create an additional team for students yet to be assign a team
def createTeamForRemainingStudent(totalNumStudents, teamSize, teamList, icount, maleList, femaleList):
    remainingStudent = totalNumStudents % teamSize
    if remainingStudent !=0:
        if isinstance(icount, list):
            mcount=icount[0]
            fcount=icount[1]
        else:
            mcount = icount
            fcount = icount
        teamList.append([])
        while fcount < len(femaleList):
            teamList[-1].append(femaleList[fcount]) 
            fcount += 1
        while mcount < len(maleList):
            teamList[-1].append(maleList[mcount]) 
            mcount += 1
    return teamList

# A function to sort all students beloning to the same tutorial group by gender
def assignTeamsByGender(teamSize, studentList, tutorialGroupNum):

    # tgrp = all students belonging to the tutorialGroupNum
    # Create a malelist to store all male students and a female list to store all female students
    # totalNumStudents = total number of students belonging to the tutorial group
    tgrp = retrieveTutorialGroup(studentList, tutorialGroupNum)
    totalNumStudents=0
    maleList = []
    femaleList = []
    for st in tgrp:
        if st.gender == 'Female':
            femaleList.append(st)
        elif st.gender == 'Male':
            maleList.append(st)
        totalNumStudents+=1
    
    # numOfTeams = the number of teams that can satisfy teamSize. 
    # Create a list of teams based on numOfTeams
    numOfTeams = totalNumStudents // teamSize 
    teamList = []
    for _ in range (numOfTeams): 
        teamList.append([])

    # Based on teamSize, determine max number of m/f students that can be added in 1:1 ratio to a team. 
    # In a team of 5, max number will be 2m:2f which satisfy 1:1 ratio
    split = teamSize // 2 

    # Return an empty teamList if theres 0 students to add in the first place
    if len(maleList) == 0 and len(femaleList) == 0:
        return teamList
    
    # Return sorted teamList
    else:

        # icount - A counter used to access each student in male/female list
        icount=0 

        # For each created team in teamList
        for nTeam in teamList:

            # Setting 1m:1f ratio for sorting and adding of students from male/female list into the created teams in teamList
            for _ in range(split):

                # Return teamList if theres no more students to add from both male/female list
                if icount >= len(maleList) and icount >= len(femaleList):
                    return teamList
                
                # End 1m:1f ratio if theres no more female students to add
                # Fill up all teams with male students if teamSize not met
                # createTeamForRemainingStudent() checks for students yet to be assign a team. 
                # If yes, create a new team and add them to it
                # Return sorted teamList
                elif icount >= len(maleList):
                    for nTeam in teamList:
                        while len(nTeam) != teamSize:
                            if icount < len(femaleList):
                                nTeam.append(femaleList[icount])
                                icount+=1
                    teamList = createTeamForRemainingStudent(totalNumStudents, teamSize, teamList, icount, maleList, femaleList)
                    return teamList
                
                # End 1m:1f ratio if theres no more male students to add
                # Fill up all teams with female students if teamSize not met
                # createTeamForRemainingStudent() checks for students yet to be assign a team. 
                # If yes, create a new team and add them to it
                # Return sorted teamList     
                elif icount >= len(femaleList):
                    for nTeam in teamList:
                        while len(nTeam) != teamSize:
                            if icount < len(maleList):
                                nTeam.append(maleList[icount])
                                icount+=1
                    teamList = createTeamForRemainingStudent(totalNumStudents, teamSize, teamList, icount, maleList, femaleList)
                    return teamList
                
                # Sort and add students according to 1m:1f ratio 
                else:
                    nTeam.append(maleList[icount])
                    nTeam.append(femaleList[icount])
                    icount += 1

        # After finished sorting and adding students according to 1m:1f ratio
        # There are still students of both genders yet to be assign a team
        # Fill up all teams with male & female students if teamSize not met
        # Order of adding male/female students does not matter but for simplicity, we start with one gender first.
        # createTeamForRemainingStudent() checks for students yet to be assign a team. 
        # If yes, create a new team and add them to it
        # Return sorted teamList
        mcount=icount
        fcount=icount
        for nTeam in teamList:
            while len(nTeam) != teamSize:
                if fcount < len(femaleList):
                    nTeam.append(femaleList[fcount])
                    fcount+=1
                elif mcount < len(maleList):
                    nTeam.append(maleList[mcount]) 
                    mcount+=1
        icount = [mcount, fcount]
        teamList = createTeamForRemainingStudent(totalNumStudents, teamSize, teamList, icount, maleList, femaleList)

    return teamList

#### Second Algorithm - Sort By CGPA

- Creates empty list for teams
- Uses retrieveTutorialGroup function to get the list of students from a specific tutorial group
- Calculates number of teams needed
- In the list created, create empty list for total teams calculated.
- Since the list of students retrieved is already sorted by CGPA (Highest to Lowest)
    * Proceed to distribute students into teams by rows in our case
    * For 10 teams of 5, it will go like:
        * Step 1: Distributes first 10 students aka top 10 students across all teams
        * Step 2: Distributes next 10 in reverse order & go to step 3
        * Step 3: Repeat step 2 unless all students are assigned to a team
    * This algorithm ensures that each team gets:
        * 1 top student (rank 0-9)
        * 1 high-mid student (rank 10-19)
        * 1 mid student (rank 20-29)
        * 1 low-mid student (rank 30-39)
        * 1 bottom student (rank 40-49)
- Finally return the list of teams with appended data

In [ ]:
def assignTeamsByCGPA(studentsPerTeam, tgrp):
    # Initialize the required lists for the function
    teams = []
    tutorialList = retrieveTutorialGroup(studentList, tgrp)

    # We add an additional count of a team - 1 to to find the max number of teams including possible outliers (e.g outliers of 3 / 4 man groups)
    numberOfTeams = (len(tutorialList) + studentsPerTeam - 1) // studentsPerTeam

    # Main while loop to sort each tutorial group
    for i in range(numberOfTeams):
        teams.append([])

    # We make use of the snaking pattern when sorting through the list matrix going from left to right, then right to left, etc...
    # This will ensure a balance of highs, mids and lows for a balanced distribution
    row = 0
    for i in range(len(tutorialList)):
        # Starting from the top of the sorted tutorial list, append from the current i: index in the list to the respective teams
        # This spreads all the highs / lows evenly for the first distribution from left to right
        if row % 2 == 0: # Check if the current row in the list is even
            teamNum = (i % numberOfTeams) + 1 # Modulo + 1 ensures that the index will remain between 1 and number of teams for the index
            
        else: # Else the row is considered odd
            # On the 'second' append iteration, we now insert from right to left
            # Additionally, the teamNum will also be in reverse right to left - This will result in the snaking pattern
            teamNum = numberOfTeams - (i % numberOfTeams)
        
        # We append the indexed student into the respective teams via the snake sorting pattern
        teams[teamNum - 1].append(tutorialList[i])
        
        # Increment the row after every team has been mutilated with new data
        if (i + 1) % numberOfTeams == 0:
            row += 1
    
    # We finally return the teams dict only after reducing the deviation by calling 'reduceTeamDist'
    return teams

#### Third Algorithm - Sort By Faculty

In [ ]:
# A function to purely assign teams by faculty only without other restrictions
def assignTeamsByFaculty(teamSize, studentList, tutorialGroup, maxFac):
    teams = []
    current_group = []
    faculty_count = {}
    studentsInTgrp = retrieveTutorialGroup(studentList, tutorialGroup)

    for student in studentsInTgrp:
        faculty = student.faculty
    
        count = faculty_count.get(faculty, 0)

        if count < maxFac:
            current_group.append(student)
            faculty_count[faculty] = count + 1
        else: #if faculty limited reached, start a new group
            teams.append(current_group)
            current_group = [student]
            faculty_count = {faculty: 1}

        #if group is full, start a new one and reset count
        if len(current_group) == teamSize:
            teams.append(current_group)
            current_group = []
            faculty_count = {}

    #add any remaining students in the last group
    if current_group:
        teams.append(current_group)

    for index, group in enumerate(teams, start=1):
        print(f"\nGroup {index}:")
        for student in group:
            print(f'{student.name} and {student.faculty}')
    
    return teams

# A function that returns the number of faculties in a given team
def facultiesInTeam(team):
    # We make use of sets to represent the number of unique faculties in a team
    # This is because items / elements in a set in python are always unique (No 2 items are the same),
    # we do this because we only want to know the unique number and not the total number of faculties
    faculties = {student.faculty for student in team}
    return len(faculties)

# A function that returns a boolean value based on whether every team in a tutorial group is "Balanced in terms of Faculties"
def checkFacultyBalanced(teams, minFacultyPerTeam):
    # A set containing unique entries of faculties in each team
    uniqueFacultiesPerTeam = {}
    
    for team in teams:
        # We add each faculty in a team to the set to find its uniqueness
        uniqueFacultiesPerTeam = {student.faculty for student in team}
        
        # We then check if it meets the minimum number of faculties in a team, if not we return false
        # Our goal here is every team must be balanced, only then we return true so if even one fails, we return false
        if len(uniqueFacultiesPerTeam) < minFacultyPerTeam:
            return False
    
    return True

# A function that is utilized the the swapping algorithm to check if the current swap is an improvement over the original teams
def checkFacultyUniqueness(teamOne, teamTwo, minFac, maxFac):    
    # In the case where the size of the two teams do not match
    # Important because we want to avoid the case of a outlier 5 Man Team compare with a 2 Man Team
    if len(teamOne) != len(teamTwo):
        return False
    
    # Get the count of uniqe faculties in each team 
    teamOneFaculties = facultiesInTeam(teamOne)
    teamTwoFaculties = facultiesInTeam(teamTwo)
    
    # If either team does not meet the mininum number of faculties required in a team, we return false
    if teamOneFaculties < minFac or teamTwoFaculties < minFac:
        return False
    
    # Else, we check if the current swapped members form a team that has met the minimum number of faculties required in a team
    if teamOneFaculties < maxFac or teamTwoFaculties < maxFac:
        return True
    
    return False

#### Distribution Reduction Algorithm - Conditional Swapping

------ CGPA Distribution - 0 ------



------ Gender Distribution - 1 ------



------ Faculty Distribution - 2 ------

The function reduces the faculty distribution between each team ensuring that they are well balanced
There should not be a team that contains a dominant faculty e.g., A group of 5 having 4 CCDS students and 1 NBS student

However, in cases where number of students in teams equals 2 or 3 students:
2 unique faculties in such teams are still justified as balanced e.g., 2 CCDS and 1 NBS / 1 CCCDS and 1 NBS

For team sizes with 4 students or more, minimally we want 3 faculties minimum for a balanced distribution
E.g., 2 CCDS, 1 NBS and 1 SoH / 1 CCDS, 1 NBS, 1 SoH, 1 EEE

Assume that we are creating teams of 4 students and more
We make use of python sets to define "faculty uniqueness" within each team, if the len(set) < 3 we will attempt to reorganize the teams by swapping students around

------ Faculty Distribution - 3 ------



In [ ]:
# ------ Distribution Code ------

# Sort Conditions:
# 0: Reduce team distribution by considering only cgpa (Extreme Case Swap)
# 1: Reduce team distribution by considering both cgpa and gender (Extreme Case Swap)
# 2: Reduce team distribution by considering cgpa, gender and faculty (Extreme Case Swap)
# 3: Reduce team distribution by considering cgpa, gender and faculty (Random Swap)

# For Extreme Case Swaps:
# The number of swap attempts needed is > 30 and < 100 for a more balanced and resource efficient program

# For Random Swap:
# The number of random swap attempts needed is > 500 and < 3000. The program is less efficient, however it produces the smallest distribution due to its random diversity

# A function to attempt to reduce the distrubtion in the teams between the largest average CGPA and lowest average CGPA
# We do a random swap to attempt to reduce the distribution
def reduceTeamDist(teams, sortCondition, minFac, maxFac, randAttempts = 50):
    tempTeams = teams[:] # We do a copy of "teams" argument to prevent referencing the old reference pointer
    
    # Faculty Balanced Flag
    facBalanced = False
    
    # Attempt a fixed number of random swaps
    currentAttempts = 0
    while currentAttempts < randAttempts: # Attempt the random swap at n attempts
        # Here we want to create a list that loops through each team
        # and uses the sum method to get the total cgpa sum of each team in the tutorial group
        
        # This is a list of cgpas per team in a tutorial group
        teamCGPAList = [sum(student.cgpa for student in team) / len(team) for team in tempTeams]
        
        # Based on that list, we want to do a internal swap using two extremes
        # Using this swapping method, each time we perform a improved swap, the extremes will change and the idea is that the 
        # more times we perform these swaps, the gap between these two extremes will also get smaller and smaller
        firstIndex = teamCGPAList.index(max(teamCGPAList)) # We find the index of team with the highest extreme
        secIndex = teamCGPAList.index(min(teamCGPAList)) # We find the index team with the lowest extreme
        
        # Reference the two randomly selected teams for comparison and swapping
        teamOne = tempTeams[firstIndex]
        teamTwo = tempTeams[secIndex]
        
        # Only for sort condition 3 (Swap by Random)
        randIntOne = random.randint(0, len(teams) - 1)
        randIntTwo = random.randint(0, len(teams) - 1)
        
        # Ensure that the second random integer is not the same as the first
        while randIntOne != randIntOne:
            randIntTwo = random.randint(0, len(teams) - 1)
        
        # We change the teamOne and teamTwo datasets to randomized teams instead of fixed extremes for sortCondition 3
        if sortCondition == 3:
            teamOne = tempTeams[randIntOne]
            teamTwo = tempTeams[randIntTwo]
        
        # Calculate CGPA difference between the two teams
        teamOneSumCgpa = sum(student.cgpa for student in teamOne)
        teamTwoSumCgpa = sum(student.cgpa for student in teamTwo)
        
        # Save the current best average CGPA difference between the original two teams
        bestDifference = abs((teamOneSumCgpa / len(teamOne)) - (teamTwoSumCgpa / len(teamTwo)))
        
        # We use a switch statement to switch between the different sorting conditions to allow for flexibility in the algorithm
        match sortCondition:
            case 0: # Swap purely by CGPA only
                for st1Index, st1 in enumerate(teamOne):
                    for st2Index, st2 in enumerate(teamTwo):
                        # We perform a 1-for-1 swap in positions of the two students, swapping them between the two teams
                        teamOne[st1Index], teamTwo[st2Index] = teamTwo[st2Index], teamOne[st1Index]
                        
                        # Because we swapped the two students, we remove the cgpa of the old student and add the cgpa of the new student in each team
                        teamOneSumCgpa = (teamOneSumCgpa - st1.cgpa) + st2.cgpa
                        teamTwoSumCgpa = (teamTwoSumCgpa - st2.cgpa) + st1.cgpa
                        
                        # Save the current best average CGPA difference between the original two teams
                        newDifference = abs((teamOneSumCgpa / len(teamOne)) - (teamTwoSumCgpa / len(teamTwo)))
                        
                        # We want to see if the new difference is smaller or not, if not we revert it back and try again with the other students in the two teams
                        if newDifference < bestDifference:
                            bestDifference = newDifference
                        else:
                            # Revert back
                            teamOneSumCgpa = (teamOneSumCgpa + st1.cgpa) - st2.cgpa
                            teamTwoSumCgpa = (teamTwoSumCgpa + st2.cgpa) - st1.cgpa
                            teamOne[st1Index], teamTwo[st2Index] = teamTwo[st2Index], teamOne[st1Index]

            case 1: # Swap purely by CGPA and Gender only
                for st1Index, st1 in enumerate(teamOne):
                    for st2Index, st2 in enumerate(teamTwo):
                        # We perform a 1-for-1 swap in positions of the two students, swapping them between the two teams
                        teamOne[st1Index], teamTwo[st2Index] = teamTwo[st2Index], teamOne[st1Index]
                        
                        # Because we swapped the two students, we remove the cgpa of the old student and add the cgpa of the new student in each team
                        teamOneSumCgpa = (teamOneSumCgpa - st1.cgpa) + st2.cgpa
                        teamTwoSumCgpa = (teamTwoSumCgpa - st2.cgpa) + st1.cgpa
                        
                        # Save the current best average CGPA difference between the original two teams
                        newDifference = abs((teamOneSumCgpa / len(teamOne)) - (teamTwoSumCgpa / len(teamTwo)))
                        
                        # We want to see if the new difference is smaller or not, if not we revert it back and try again with the other students in the two teams
                        if st1.gender == st2.gender and newDifference < bestDifference:
                            bestDifference = newDifference
                        else:
                            # Revert back
                            teamOneSumCgpa = (teamOneSumCgpa + st1.cgpa) - st2.cgpa
                            teamTwoSumCgpa = (teamTwoSumCgpa + st2.cgpa) - st1.cgpa
                            teamOne[st1Index], teamTwo[st2Index] = teamTwo[st2Index], teamOne[st1Index]

            case 2: # Swap by CGPA, Gender and Faculty
                # Save the most balanced teams based on faculty first
                if not facBalanced:
                    if checkFacultyBalanced(tempTeams, minFac):
                        facBalanced = True
                    else:
                        for st1Index, st1 in enumerate(teamOne):
                            for st2Index, st2 in enumerate(teamTwo):
                                # We perform a 1-for-1 swap in positions of the two students, swapping them between the two teams
                                teamOne[st1Index], teamTwo[st2Index] = teamTwo[st2Index], teamOne[st1Index]
                                
                                # Because we swapped the two students, we remove the cgpa of the old student and add the cgpa of the new student in each team
                                teamOneSumCgpa = (teamOneSumCgpa - st1.cgpa) + st2.cgpa
                                teamTwoSumCgpa = (teamTwoSumCgpa - st2.cgpa) + st1.cgpa
                                
                                # Save the current best average CGPA difference between the original two teams
                                newDifference = abs((teamOneSumCgpa / len(teamOne)) - (teamTwoSumCgpa / len(teamTwo)))
                                
                                if checkFacultyUniqueness(teamOne, teamTwo, minFac, maxFac) and st1.gender == st2.gender and newDifference < bestDifference:
                                    bestDifference = newDifference
                                else:
                                    # Revert back
                                    teamOneSumCgpa = (teamOneSumCgpa + st1.cgpa) - st2.cgpa
                                    teamTwoSumCgpa = (teamTwoSumCgpa + st2.cgpa) - st1.cgpa
                                    teamOne[st1Index], teamTwo[st2Index] = teamTwo[st2Index], teamOne[st1Index]
                        
                        currentAttempts += 1
                        continue # Ignore the rest of the code below and try again
                
                return reduceTeamDist(tempTeams, 4, minFac, maxFac)

            case 3: # Randomly swap by CGPA, Gender and Faculty (With faculty as the focus)
                # Save the most balanced teams based on faculty first
                if not facBalanced:
                    if checkFacultyBalanced(tempTeams, minFac):
                        facBalanced = True
                    else:
                        for st1Index, st1 in enumerate(teamOne):
                            for st2Index, st2 in enumerate(teamTwo):
                                # We perform a 1-for-1 swap in positions of the two students, swapping them between the two teams
                                teamOne[st1Index], teamTwo[st2Index] = teamTwo[st2Index], teamOne[st1Index]
                                
                                # Because we swapped the two students, we remove the cgpa of the old student and add the cgpa of the new student in each team
                                teamOneSumCgpa = (teamOneSumCgpa - st1.cgpa) + st2.cgpa
                                teamTwoSumCgpa = (teamTwoSumCgpa - st2.cgpa) + st1.cgpa
                                
                                # Save the current best average CGPA difference between the original two teams
                                newDifference = abs((teamOneSumCgpa / len(teamOne)) - (teamTwoSumCgpa / len(teamTwo)))
                                
                                if checkFacultyUniqueness(teamOne, teamTwo, minFac, maxFac) and st1.gender == st2.gender and newDifference < bestDifference:
                                    bestDifference = newDifference
                                else:
                                    # Revert back
                                    teamOneSumCgpa = (teamOneSumCgpa + st1.cgpa) - st2.cgpa
                                    teamTwoSumCgpa = (teamTwoSumCgpa + st2.cgpa) - st1.cgpa
                                    teamOne[st1Index], teamTwo[st2Index] = teamTwo[st2Index], teamOne[st1Index]
                        
                        currentAttempts += 1
                        continue # Ignore the rest of the code below and try again
                
                if facBalanced:
                    for st1Index, st1 in enumerate(teamOne):
                        for st2Index, st2 in enumerate(teamTwo):
                            # We perform a 1-for-1 swap in positions of the two students, swapping them between the two teams
                            teamOne[st1Index], teamTwo[st2Index] = teamTwo[st2Index], teamOne[st1Index]
                            
                            # Because we swapped the two students, we remove the cgpa of the old student and add the cgpa of the new student in each team
                            teamOneSumCgpa = (teamOneSumCgpa - st1.cgpa) + st2.cgpa
                            teamTwoSumCgpa = (teamTwoSumCgpa - st2.cgpa) + st1.cgpa
                            
                            # Save the current best average CGPA difference between the original two teams
                            newDifference = abs((teamOneSumCgpa / len(teamOne)) - (teamTwoSumCgpa / len(teamTwo)))
                            
                            if st1.faculty == st2.faculty and st1.gender == st2.gender and newDifference < bestDifference:
                                bestDifference = newDifference
                            else:
                                # Revert back
                                teamOneSumCgpa = (teamOneSumCgpa + st1.cgpa) - st2.cgpa
                                teamTwoSumCgpa = (teamTwoSumCgpa + st2.cgpa) - st1.cgpa
                                teamOne[st1Index], teamTwo[st2Index] = teamTwo[st2Index], teamOne[st1Index]

            case 4: # A recursive case to help reduce the faculty swap distribution further (Referenced locally and recursively)
                for st1Index, st1 in enumerate(teamOne):
                    for st2Index, st2 in enumerate(teamTwo):
                        # We perform a 1-for-1 swap in positions of the two students, swapping them between the two teams
                        teamOne[st1Index], teamTwo[st2Index] = teamTwo[st2Index], teamOne[st1Index]
                        
                        # Because we swapped the two students, we remove the cgpa of the old student and add the cgpa of the new student in each team
                        teamOneSumCgpa = (teamOneSumCgpa - st1.cgpa) + st2.cgpa
                        teamTwoSumCgpa = (teamTwoSumCgpa - st2.cgpa) + st1.cgpa
                        
                        # Save the current best average CGPA difference between the original two teams
                        newDifference = abs((teamOneSumCgpa / len(teamOne)) - (teamTwoSumCgpa / len(teamTwo)))
                        
                        if st1.faculty == st2.faculty and st1.gender == st2.gender and newDifference < bestDifference:
                            bestDifference = newDifference
                        else:
                            # Revert back
                            teamOneSumCgpa = (teamOneSumCgpa + st1.cgpa) - st2.cgpa
                            teamTwoSumCgpa = (teamTwoSumCgpa + st2.cgpa) - st1.cgpa
                            teamOne[st1Index], teamTwo[st2Index] = teamTwo[st2Index], teamOne[st1Index]

            case _: # A default case in case the user specifies a non-extistant sort condition
                print("Invalid sort condition, please check again")

        # If this current swap selection doesn't work out, we try again until satisfied
        currentAttempts += 1

    # Once satisfied or attempts finished, we return the newest updated team list with a lower deviation
    return tempTeams

#### Output Sorted CSV with New 'Assigned Team' Column

In [ ]:
# Output a new csv file with the new given list of students
def outputCSV(data):
    headers = ['Tutorial Group','Student ID','School','Name','Gender','CGPA','Team Assigned']
    
    rows = []
    
    for tgrp in data:
        for teams in tgrpsTeamsList[tgrp]:
            for student in teams:
                studDetails = f'G-{student.tutorialGrp},{student.studID},{student.faculty},{student.name},{student.gender},{student.cgpa}'
                rows.append(studDetails)

    file = open('new_teams.csv', 'w')

    header_line = ', '.join(str(h) for h in headers)
    file.write(header_line + '\n')

    for row in rows:
        file.write(row + '\n')

    file.close()

#### Main Body Code

In [ ]:
def sortAndAssignTeams(sortCondition, teamSize):
    numOfTgrps = getNumOfTgrps(studentList)

    # Specify the max count of unique faculties in a team
    # E.g., Students in a team = 5
    #       Max unique faculties = 5
    #       Min unique faculties = 4
    minFacultyPerTeam = teamSize - 1
    maxFacultiesPerTeam = teamSize
    
    match(sortCondition):
        case 1: # Sort only by Gender
            for i in range(0, numOfTgrps):
                teams = assignTeamsByGender(teamSize, studentList, i + 1)
                tgrpsTeamsList[i + 1] = teams

        case 2: # Sort only by Faculty
            for i in range(0, numOfTgrps):
                teams = assignTeamsByFaculty(teamSize, studentList, i + 1, maxFacultiesPerTeam)
                tgrpsTeamsList[i + 1] = teams

        case 3: # Sort only by CGPA
            for i in range(0, numOfTgrps):
                teams = assignTeamsByCGPA(teamSize, i + 1) # We first create the teams using the assignTeamsByCGPA function
                teams = reduceTeamDist(teams, 1, minFacultyPerTeam, maxFacultiesPerTeam) # Then we reduce the distribution
                tgrpsTeamsList[i + 1] = teams
            
        case 4:
            for i in range(0, numOfTgrps):
                teams = assignTeamsByGender(teamSize, studentList, i + 1)
                teams = reduceTeamDist(teams, 3, minFacultyPerTeam, maxFacultiesPerTeam)
                tgrpsTeamsList[i + 1] = teams
            
        case 5:
            for i in range(0, numOfTgrps):
                teams = assignTeamsByGender(teamSize, studentList, i + 1)
                teams = reduceTeamDist(teams, 3, minFacultyPerTeam, maxFacultiesPerTeam)
                tgrpsTeamsList[i + 1] = teams

        case 'RandomSwap': # This case is only for comparison of sorting algorithms
            for i in range(0, numOfTgrps):
                teams = assignTeamsByGender(teamSize, studentList, i + 1)
                teams = reduceTeamDist(teams, 3, minFacultyPerTeam, maxFacultiesPerTeam, 500)
                tgrpsTeamsList[i + 1] = teams

        case _:
            return False
        
    outputCSV(tgrpsTeamsList)
    return True

In [ ]:
# A IPywidget integer slider that allows the user to pick an integer from a range of 4 - 10
teamSizeSlider = widgets.IntSlider(
    value = 5,
    min = 4,
    max = 10,
    description='Team Size',
    layout=widgets.Layout(width='25%', height='30px')
)

sortingCondition = widgets.ToggleButtons(
    layout=widgets.Layout(margin='10px'),
    options=['Sort by Gender', 'Sort by Faculty', 'Sort by CGPA', 'Sort by Gnd & CGPA', 'Sort by Everything'],
    tooltips=['Sort only by gender ratios', 'Sort only by faculty ratios', 'Sort only by CGPA Distribution', 'Sort by both Gender and CGPA', 'Sort by Gender, CGPA and Faculty'],
    description='Select Your Sorting Condition: (Hover for more info)'
)

# The submit button that enables the user to start the sorting algorithm
startSortButton = widgets.Button(
    description='Start Sorting',
    tooltip='Submit',
    button_style='warning',
    layout=widgets.Layout(width='25%', height='50px', margin='10px')
)

textBox = widgets.HTML(
    description='Sort Progress: ',
    value='Nothing yet...',
    layout={'border': '2px solid black'}
)

resetButton = widgets.Button(
    description='Reset Parameters',
    tooltip='Submit',
    button_style='danger',
    layout=widgets.Layout(width='25%', height='50px', margin='10px')
)

# Display all the IPywidget elements
display(teamSizeSlider)
display(sortingCondition)
display(startSortButton)
display(textBox)
display(resetButton)

# Validate that there is minimally one team that can be created from each tutorial group
# Else display a warning
def validateParameters(button):
    teamSize = teamSizeSlider.value
    studentsPerTgrp = {}
    sortBy = 0 # 0 means illegal sort condition
    
    # We do a switch case here to match the sorting condition with an integer for easier referencing in the backend code
    match(sortingCondition.value):
        case 'Sort by Gender':
            sortBy = 1
        case 'Sort by Faculty':
            sortBy = 2
        case 'Sort by CGPA':
            sortBy = 3
        case 'Sort by Gnd & CGPA':
            sortBy = 4
        case 'Sort by Everything':
            sortBy = 5
        case _:
            sortBy = 0
    
    # We use a for loop and dictionaries to count the number of students in each tutorial group
    for student in studentList:
        studentsPerTgrp[student.tutorialGrp] = 0
        
    for student in studentList:
        studentsPerTgrp[student.tutorialGrp] += 1
    
    # We check that the selected teamSize is smaller than the smallest number of students
    # from the list of total students from each and every tutorial group.
    if teamSize <= min(studentsPerTgrp.values()):
        button.button_style = 'success'
        button.description = 'Sorting Now!'
        button.disabled = True # Stop the user from spamming the sort button
        
        # Since successful, we start calling the sorting algorithm based on specified sorting requirements
        done = sortAndAssignTeams(sortBy, teamSize)
        
        # Show a sorting in progress statement
        textBox.value = '\nSorting in progres...\n'
        
        if done:
            textBox.value = "\nYour file is done sorting! Please check your local folder, thank you.\n"
            resetButton.disabled = False
        else:
            button.disabled = False
            textBox.value = "\nThere was an error sorting the students, please try again!\n"
    else:
        button.button_style = 'danger'
        button.description = 'Team size too big!'

def resetParameters(button):
    button.disabled = True
    startSortButton.disabled = False
    startSortButton.button_style = 'warning'
    startSortButton.description = 'Start Sorting'
    textBox.value = 'Nothing yet...'
    teamSizeSlider.value = 5

# A onClick observer is placed on the buttons, once clicked it performs a callback onto the validateParameters / resetParameters function
# If the teamSize meets the requirements, the sort begins else user has to choose a smaller team size
startSortButton.on_click(validateParameters)
resetButton.on_click(resetParameters) # Reset all the parameters to allow user to sort again

#### Post-sort Analysis - Plotly Graphs

#### A sorting algorithm comparison

In [ ]:
# Only uncomment the code during presentation
# sortAndAssignTeams('RandomSwap', 5)

In [ ]:
# Only uncomment the code during presentation
# sortAndAssignTeams(5, 5)